## Partie pré-traitement des données communes

Dans ce notebook, nous récuppérons et traitons des données de trois ensembles de données différents. Cela nous permettra de collecter des informations sur les communes concernant:
- La géographie (Département, lat, lng ..)
- La démographie (denisté, population ..)
- Le standard de vie (revenue_mean ..)
   
Ces données seront utilisées ultérieurement lors de l'étape d'analyse des données.

L'objectif est d'avoir une seule Dataframe globale qui regroupe les données mentionnées en haut.

## 1.Importation de librairies nécessaires

In [ ]:
import numpy as np
import pandas as pd 
from statsmodels.stats.weightstats import DescrStatsW # For dealing with weighted stats

# Librairies de DataViz Matplotlib 

import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import seaborn as sns

import time

## 2. Importation and pre-traitement "geo-data"

In [ ]:
# Charge les données de communes géographiques
# La dataframe présente plusieurs colonnes avec des valeurs NaN, on va sélectionner que les colonnes nécessaires

communes= pd.read_csv('datasets/correspondances-code-insee-code-postal.csv', sep=';',
                      usecols= ['insee_com','insee_com','nom_comm','superficie','geo_point_2d','code_dept','code_reg'],
                     dtype= {'insee_com':'object'})

communes.rename(columns={'insee_com':'Code INSEE'},inplace=True)

print(communes.shape)
communes.head()

(1300, 6)


,Code INSEE,nom_comm,superficie,geo_point_2d,code_dept,code_reg
0,91645,VERRIERES-LE-BUISSON,999.0,"48.75044312,2.25171297214",91,11
1,77133,COURCELLES-EN-BASSEE,1082.0,"48.4125606521,3.05294050556",77,11
2,91378,MAUCHAMPS,313.0,"48.5272680908,2.19718165044",91,11
3,77243,LAGNY-SUR-MARNE,579.0,"48.8730701858,2.70978081313",77,11
4,94003,ARCUEIL,232.0,"48.8058803597,2.33351024984",94,11


In [ ]:
# Convertion des surface 'communes' en km2
communes['superficie_km2']=communes['superficie']/100
communes.drop(['superficie'], axis=1, inplace=True)

# Splitting des coordonées géo en 2 colonnes 'lat' and 'lng'
communes[['lat','lng']]=communes.geo_point_2d.str.split(',', expand=True)
communes.drop(['geo_point_2d'], axis=1, inplace=True)
communes.head(3)

,Code INSEE,nom_comm,code_dept,code_reg,superficie_km2,lat,lng
0,91645,VERRIERES-LE-BUISSON,91,11,9.99,48.75044312,2.25171297214
1,77133,COURCELLES-EN-BASSEE,77,11,10.82,48.4125606521,3.05294050556
2,91378,MAUCHAMPS,91,11,3.13,48.5272680908,2.19718165044


In [ ]:
communes.describe(include='all')

,Code INSEE,nom_comm,code_dept,code_reg,superficie_km2,lat,lng
count,1300,1300,1300.000000,1300.0,1300.000000,1300,1300
unique,1300,1296,NaN,NaN,NaN,1300,1300
top,95491,MONDREVILLE,NaN,NaN,NaN,49.0058198768,2.1221470247
freq,1,2,NaN,NaN,NaN,1,1
mean,NaN,NaN,83.365385,11.0,9.275046,NaN,NaN
std,NaN,NaN,7.746337,0.0,7.757466,NaN,NaN
min,NaN,NaN,75.000000,11.0,0.090000,NaN,NaN
25%,NaN,NaN,77.000000,11.0,4.745000,NaN,NaN
50%,NaN,NaN,78.000000,11.0,7.675000,NaN,NaN
75%,NaN,NaN,91.000000,11.0,11.732500,NaN,NaN


## 3. Importation et pré-traitement des données démographiques 


In [ ]:
# Import des données age, population
# La dataframe présente plusieurs colonnes avec des valeurs NaN, on va sélectionner que les colonnes qu'on aura besoin datatype = {'com_code':'object','com_type':'object','popmun_age':'float64',
            'popmun_sexe':'object','popmun_nb':'float64'}
population= pd.read_csv('datasets/t-popmun-2016-com.csv', encoding = "ISO-8859-1",sep=',',
                        usecols= ['com_code','com_type','popmun_age','popmun_sexe','popmun_nb'],
                        dtype=datatype)
print(population.shape[0])
population.head()

7069596


,com_code,com_type,popmun_age,popmun_sexe,popmun_nb
0,01001,COM,0.0,F,3.0
1,01001,COM,0.0,M,13.0
2,01001,COM,1.0,F,5.0
3,01001,COM,1.0,M,1.0
4,01001,COM,2.0,F,6.0


Préparation des données démographique :
- On essaie de grouper la colonne age (colonne : popmun_age) par le Code INSEE de commune(colonne : com_code) 

- On prendra compte de la somme de personnes total, ce qui représente la population (colonne : popmun_nb)

In [ ]:
#On va construire une dataframe en groupant les différents age et nombre de personnes pour chaque code INSEE de la commune
# Cet dataframe va construire la base des dommées démographiques
# Dataframe sous forme multi-index
grouped_pop=population.groupby(['com_code','popmun_age']).sum()
grouped_pop.head(10)

popmun_nb
com_code popmun_age           
01001    0.0              16.0
         1.0               6.0
         2.0               7.0
         3.0               5.0
         4.0               6.0
         5.0               7.0
         6.0               9.0
         7.0               8.0
         8.0              10.0
         9.0              11.0

On va utiliser le package Statsmodels pour obtenir la valeur d'âge moyenne (age_mean) et médiane (age_median) de chaque commune. 

On va regrouper tous ces informations dans une DataFrame.

Ce package a un objet qui peut traiter des observations pondérées:

In [ ]:
# Initialization des variables 
# on va construire un dictionnaire ages_stat qui regroupe les les metrics (mean, median et std) de la colonne age 
start = time.time()
age_stats={}

for com in grouped_pop.index.get_level_values(level='com_code').unique() : # Loop on all the commune number 
    wq = DescrStatsW(data=grouped_pop.loc[com].index, weights=grouped_pop.loc[com].values ) # Use a DescrStatsW object to later extract the mean and median values
    age_stats[com] = [wq.mean,wq.std,wq.quantile(0.5, return_pandas = False)[0] ,wq.nobs ] # Get and store the mean and median values in age_stats dictionnary

# Transformer le dictionnaire ages_stat en format DataFrame propre com_age
com_age=pd.DataFrame.from_dict(age_stats, orient='index',columns=['age_mean', 'age_std', 'age_median', 'population'])
com_age.reset_index(inplace=True)
com_age.rename(columns={'index': "Code INSEE"}, inplace=True)

end = time.time()
temps=end - start
print('Temps de traitement :',temps, 'secondes ')

C:\Users\Administrateur\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:133: RuntimeWarning: invalid value encountered in double_scalars
  return self.sum / self.sum_weights


Temps de traitement : 87.95080614089966 secondes 


In [ ]:
# explorer la version finale des données démographique
print(com_age.shape)
com_age.head()

(34998, 5)


,Code INSEE,age_mean,age_std,age_median,population
0,01001,40.814863,23.714339,44.0,767.0
1,01002,38.740741,23.894028,38.0,243.0
2,01004,38.397455,23.933946,37.0,14081.0
3,01005,39.125199,23.160941,41.0,1671.0
4,01006,47.000000,22.272690,49.0,110.0


In [ ]:
#com_age.to_csv(r'com_age.csv')
com_age= pd.read_csv(r'com_age.csv', index_col=0)

Quelques communes on des valeurs NaN pour age_mean et age_std. 
Ce qui veut dire qu'ils existent des communes sans habitants, on va essayer de lister ces communes : 

In [ ]:
# uninhabited_com est une liste des communes sans habitants
uninhabited_com=[]
for column in com_age.columns[1:com_age.columns.shape[0]] :
    for i in range(0,com_age[column].shape[0]) :
        if np.isnan(com_age[column][i]) :
            uninhabited_com.append(com_age['Code INSEE'][i])
uninhabited_com=list(dict.fromkeys(uninhabited_com))        
print('After a quick analysis we can see that those area code are empty, and have no population :',
      "\n", uninhabited_com)

# Lire le dataframe qui présente les communes sans habitans
com_age[com_age['Code INSEE'].str.contains('|'.join(uninhabited_com))]

After a quick analysis we can see that those area code are empty, and have no population : 
 ['55039', '55050', '55139', '55189', '55239', '55307']


,Code INSEE,age_mean,age_std,age_median,population
20075,55039,NaN,NaN,0.5,0.0
20086,55050,NaN,NaN,0.5,0.0
20161,55139,NaN,NaN,0.5,0.0
20204,55189,NaN,NaN,0.5,0.0
20241,55239,NaN,NaN,0.5,0.0
20300,55307,NaN,NaN,0.5,0.0


## 4. Importation et prétraitement des données de revenus de foyers

In [ ]:
# Importation des données revenues des foyers, on a choisit comme réference l'année 2017 qui présente les données les plus représentatives de l'ensemble des revenus sur 2015-2020 
# La dataframe présente plusieurs colonnes avec des valeurs NaN, on va sélectionner que les colonnes qu'on aura besoin (Code INSEE,Foyer,Nbr_p_foyer, revenus_median et revenus_inequalities)

revenues= pd.read_csv('datasets/cc_filosofi_2017_COM.CSV' , 
                      usecols=['CODGEO','NBPERSMENFISC17','NBMENFISC17','MED17','RD17'],
                      sep=';', 
                      dtype= {'CODGEO':'object',
                             'NBPERSMENFISC17':'float64',
                              'NBMENFISC17':'float64',
                              'MED17':'float64',
                              'RD17':'float64'})

revenues.rename(columns={'CODGEO':'Code INSEE',
                        'NBMENFISC17':'Foyer',
                        'NBPERSMENFISC17':'Nbr_p_Foyer',
                        'MED17':'revenue_median',
                        'RD17':'revenue_inequalities'},
                inplace=True)

print(revenues.shape)
revenues.head()

(34931, 5)


,Code INSEE,Foyer,Nbr_p_Foyer,revenue_median,revenue_inequalities
0,01001,317.0,802.0,23310.0,NaN
1,01002,107.0,258.0,24290.0,NaN
2,01004,6505.0,14567.0,19860.0,3.2
3,01005,649.0,1700.0,23370.0,NaN
4,01006,49.0,106.0,23970.0,NaN


In [ ]:
# Lire et afficher des informations meta pour chaque colonne des données revenus
pd.read_csv(r'data/meta_cc_filosofi_2017_COM.CSV' , sep=';', dtype=str).head(28)

,COD_VAR,LIB_VAR,LIB_VAR_LONG,COD_MOD,LIB_MOD,TYPE_VAR,LONG_VAR
0,NBMENFISC17,Nombre de ménages fiscaux,Nombre de ménages fiscaux,NaN,NaN,NUM,7
1,NBPERSMENFISC17,Nombre de personnes dans les ménages fiscaux,Nombre de personnes dans les ménages fiscaux,NaN,NaN,NUM,7
2,MED17,Médiane du niveau de vie (€),Médiane du niveau de vie (€),NaN,NaN,NUM,5
3,PIMP17,Part des ménages fiscaux imposés (%),Part des ménages fiscaux imposés (%),NaN,NaN,NUM,4
4,TP6017,Taux de pauvreté-Ensemble (%),Taux de pauvreté-Ensemble (%),NaN,NaN,NUM,4
5,TP60AGE117,Taux de pauvreté-moins de 30 ans (%),Taux de pauvreté des personnes dans les ménage...,NaN,NaN,NUM,4
6,TP60AGE217,Taux de pauvreté-30 à 39 ans (%),Taux de pauvreté des personnes dans les ménage...,NaN,NaN,NUM,4
7,TP60AGE317,Taux de pauvreté-40 à 49 ans (%),Taux de pauvreté des personnes dans les ménage...,NaN,NaN,NUM,4
8,TP60AGE417,Taux de pauvreté-50 à 59 ans (%),Taux de pauvreté des personnes dans les ménage...,NaN,NaN,NUM,4
9,TP60AGE517,Taux de pauvreté-60 à 74 ans (%),Taux de pauvreté des personnes dans les ménage...,NaN,NaN,NUM,4


In [ ]:
# explorer les informations des données revenus par commune/foyer (revenues) , les données démoghraphiques (com_age) et les données géographiques (communes)
print(communes.columns)
print(communes.dtypes)
print(com_age.columns)
print(com_age.dtypes)
print(revenues.columns)
print(revenues.dtypes)

Index(['Code INSEE', 'nom_comm', 'population', 'code_dept', 'code_reg',
       'superficie_km2', 'lat', 'lng'],
      dtype='object')
Code INSEE         object
nom_comm           object
population        float64
code_dept           int64
code_reg            int64
superficie_km2    float64
lat                object
lng                object
dtype: object
Index(['Code INSEE', 'age_mean', 'age_std', 'age_median', 'population'], dtype='object')
Code INSEE     object
age_mean      float64
age_std       float64
age_median    float64
population    float64
dtype: object
Index(['Code INSEE', 'Foyer', 'Nbr_p_Foyer', 'revenue_median',
       'revenue_inequalities'],
      dtype='object')
Code INSEE               object
Foyer                   float64
Nbr_p_Foyer             float64
revenue_median          float64
revenue_inequalities    float64
dtype: object


## 5. Jointure des trois Dataframe en une seul Datafrmae global

In [ ]:
#Jointure des données géographiques et démographique - all_data
#Jointure de all_data avec les données revenues par commune/foyer

all_data=pd.merge(communes,com_age,on='Code INSEE', how='outer', indicator=True)
all_data.rename(columns={'_merge':'merge1'}, inplace=True)
all_data=pd.merge(all_data,revenues,on='Code INSEE', how='outer', indicator=True)
all_data.rename(columns={'_merge':'merge2'}, inplace=True)

# Creation d'une colonne densité - density
all_data['density']=all_data.population/all_data.superficie_km2

# Reset l'index du Dataframe global aprés jointure et explorer all_data
all_data.reset_index(drop=True, inplace=True)
all_data.head()

,Code INSEE,nom_comm,code_dept,code_reg,superficie_km2,lat,lng,age_mean,age_std,age_median,population,merge1,Foyer,Nbr_p_Foyer,revenue_median,revenue_inequalities,merge2,density
0,91645,VERRIERES-LE-BUISSON,91.0,11.0,9.99,48.75044312,2.25171297214,43.180904,25.055513,46.0,15434.0,both,6224.0,15843.0,34210.0,4.2,both,1544.944945
1,77133,COURCELLES-EN-BASSEE,77.0,11.0,10.82,48.4125606521,3.05294050556,46.291262,22.518442,51.0,219.0,both,89.0,217.0,25610.0,NaN,both,20.240296
2,91378,MAUCHAMPS,91.0,11.0,3.13,48.5272680908,2.19718165044,40.643885,22.728120,44.0,273.0,both,102.0,283.0,28080.0,NaN,both,87.220447
3,77243,LAGNY-SUR-MARNE,77.0,11.0,5.79,48.8730701858,2.70978081313,38.360069,23.270647,38.0,21264.0,both,9047.0,21301.0,23290.0,3.5,both,3672.538860
4,94003,ARCUEIL,94.0,11.0,2.32,48.8058803597,2.33351024984,37.410927,22.696750,36.0,21567.0,both,8811.0,20197.0,21750.0,4.0,both,9296.120690


In [ ]:
# sauvgarder une copie .csv de la Datraframe all_data
all_data.to_csv('datasets/all_dataset.csv', index=False)
#all_data=pd.read_csv(r'all_data.csv')

In [ ]:
# On va analyser les opérations de jointure et inspecter le Dataframe global pour s'assurer de l'exactitude des nos données
not_complete=all_data[(all_data['merge1']!='both') & (all_data['merge2']!='both')]
print('\n','Shape all_data :' , all_data.shape,
      'communes shape :', communes.shape, '\n',
      'com_age shape :', com_age.shape ,'\n',
      'revenues shape :' ,revenues.shape,'\n',
      'Rows not succesfuly merged :' ,not_complete.shape,'\n',
      'Rows in communes but not in com_age :', not_complete[not_complete.merge1=='left_only'].shape[0],'\n',
      'Rows in com_age but not in communes:', not_complete[not_complete.merge1=='right_only'].shape[0],'\n',
      'Rows in all_data but not in revenues :', not_complete[not_complete.merge2=='left_only'].shape[0],'\n',
      'Rows in revenues but not in all_data :', not_complete[not_complete.merge2=='right_only'].shape[0],'\n',
      'Rows with no Code INSEE :', all_data['Code INSEE'].isna().sum()
     )



 Shape all_data : (35011, 18) communes shape : (1300, 7) 
 com_age shape : (34998, 5) 
 revenues shape : (34931, 5) 
 Rows not succesfuly merged : (80, 18) 
 Rows in communes but not in com_age : 13 
 Rows in com_age but not in communes: 67 
 Rows in all_data but not in revenues : 80 
 Rows in revenues but not in all_data : 0 
 Rows with no Code INSEE : 0


# 6. Conclusion
Nous pouvons conclure que la jointure des données s'est bien déroulée. Toutes les statistiques d'âge et de revenus ont été ajoutées dans le DataFrame de la commune. Nous avions environ 1700 lignes supplémentaires du DataFrame de la commune, qui n'étaient pas complétées par les statistiques d'âge et de revenus. Cela peut s'expliquer par le fait que certaines de ces communes sont des territoires d'outre-mer français et ne figuraient donc pas dans les revenus initiaux ou dans le com_age DataFrame (données démograohique).